In [ ]:
! pip install plotly
! pip install pandas
! pip install matplotlib
! pip install openpyxl
! pip install nbformat 
! pip install rapidfuzz


In [ ]:
import odoo_lib as olib
import time 
import numpy as np
import pandas as pd

### **1.1 总销售额（Revenue/Sales Volume）**
- **含义**：
  - 统计每个月的销售总额，即所有销售订单的总金额。
  - 公式：`总销售额 = 每个产品的售价 × 销售数量（扣除退款或退货金额）`。
- **用途**：
  - 衡量整体销售表现。
  - 分析季节性趋势或特定促销活动对销售的影响。

In [ ]:
# Get Order Details
df_sale_order, orderline_ids = olib.fetch_all_sales__order_details()
print(f"Length of df_sale_order: {len(df_sale_order)}")
print(f"Length of orderline_ids: {len(orderline_ids)}")
df_sale_order.to_excel("data/sales_order.xlsx", index=False)
df_sale_order.sample(3)


In [ ]:
df_sale_order_lines = olib.fetch_sales_orderline_details(orderline_ids)
print(f"Total number of order lines: {len(df_sale_order_lines)}")
df_sale_order_lines.to_excel('data/sales_order_lines.xlsx')
df_sale_order_lines.sample(3)

In [ ]:
import datetime
# 数据清洗

# 被取消的orderlines
df_filtered_orderlines = df_sale_order_lines[df_sale_order_lines['state']== 'sale']

# 过去180天内的订单
today = datetime.datetime.now()
past_days = today - datetime.timedelta(days=365)
df_filtered_orderlines = df_filtered_orderlines[df_filtered_orderlines['create_date'] >= past_days]

# product_type == product
# cond = df_filtered_orderlines['product_type'] == 'product' 
# df_filtered_orderlines = df_filtered_orderlines[cond]
# qty_to_invoice == 0 and qty_to_deliver == 0
qty_to_invoice_zero = 1 # df_filtered_orderlines['qty_to_invoice'] == 0
qty_to_deliver_zero = 1 # df_filtered_orderlines['qty_to_deliver'] == 0 
product_type = df_filtered_orderlines['product_type'] != 'product' 
df_filtered_orderlines = df_filtered_orderlines[qty_to_invoice_zero & (qty_to_deliver_zero | product_type)]

df_filtered_orderlines = df_filtered_orderlines[df_filtered_orderlines['price_unit']!= 0]

# Create month, year columns, 2024-07-25 10:46:01 to 2024-07
df_filtered_orderlines['year'] = df_filtered_orderlines['create_date'].dt.strftime('%Y')
df_filtered_orderlines['month'] = df_filtered_orderlines['create_date'].dt.strftime('%Y-%m')

df_filtered_orderlines.to_excel('data/sales_order_lines.xlsx')
len(df_filtered_orderlines)


## 按订单号聚合

In [ ]:
odoo_sales_orderlines = df_filtered_orderlines.copy()
# odoo_sales_orderlines['orderline_count'] = odoo_sales_orderlines['order_number']
# 按订单号聚合订单明细.
df_agg_orderlines = odoo_sales_orderlines.groupby('order_number')  \
    .agg({'create_date': 'first', 'product_name': 'nunique', 
          'order_partner': 'first', 'salesman': 'first', 
          'price_tax': 'sum',  'price_subtotal': 'sum'}) \
        .rename(columns={'product_name': 'line_count'})  \
        .reset_index()
print(f"Length: {len(df_agg_orderlines)}")
df_agg_orderlines.sample(5)

In [ ]:
# 按销售员聚合
sales_by_salesman = odoo_sales_orderlines.groupby('salesman') \
   .agg({'price_subtotal': 'sum', 'order_number': 'nunique'}) \
   .reset_index() 

# sales_by_salesman.sort_values('amount_total', ascending=False)
sales_by_salesman

## 统计每一个客户的销售额 (不含运费，不含未结的)

### 基于 RFM 模型
RFM 是一种经典的客户分析模型，基于以下三个维度计算客户忠诚度：

1. Recency（最近一次购买时间）：
  - 距离最近一次购买时间的天数，值越小表明客户越活跃。
2. Frequency（购买频率）：
  - 客户的总购买次数，值越大表明客户越忠诚。
3. Monetary（消费金额）：
  - 客户的总消费金额，值越大表明客户贡献越高。

In [ ]:


def classify_customer(row):
    """
    Classify customers into different types based on their RFM scores.
    """
    # 高价值客户: 忠诚度评分较高，购买金额、频率较高，且最近购买时间较短。
    if row['loyalty_score'] >= 4.0:
        return 'High-Value Customers'
    # 潜在流失客户: 最近购买时间较久，频率较高，曾经是重要客户，但可能流失。
    elif row['recency_score'] in [1, 2] and row['frequency_score'] >= 4 and row['monetary_score'] >= 4:
        return 'At-Risk Customers'
    # 新客户: 最近购买时间短，但购买频率和金额较低。
    elif row['recency_score'] >= 4 and row['frequency_score'] <= 2 and row['monetary_score'] <= 2:
        return 'New Customers'
    # 其他客户
    else:
        return 'Others'
    
reference_date = pd.Timestamp.now()
# 统计每一个客户的销售额 (不含运费，不含未结的)
df_customer_sales = odoo_sales_orderlines.groupby('order_partner') \
                        .agg({'price_subtotal': sum, 'currency': 'first', 'order_number': 'nunique'}) \
                        # .reset_index()

df_customer_sales = df_customer_sales.rename(columns={'order_number': 'order_count'})

df_customer_sales = df_customer_sales.sort_values('price_subtotal', ascending=False)
df_customer_sales['avg_order_price'] = df_customer_sales['price_subtotal'] / df_customer_sales['order_count']
df_customer_sales['avg_order_price'] = df_customer_sales['avg_order_price'].round(2)

# 距离上次购买时间 (天)
last_order_days = odoo_sales_orderlines.groupby('order_partner')['create_date'].max().reset_index()
last_order_days['last_order_days'] = (reference_date - last_order_days['create_date']).dt.days
df_customer_sales = pd.merge(df_customer_sales, last_order_days[['order_partner', 'last_order_days']], on='order_partner', how='left')

# 订单金额评分
df_customer_sales['monetary_score'] = pd.qcut(df_customer_sales['price_subtotal'], 5, labels=[1, 2, 3, 4, 5])
# 订单次数评分
df_customer_sales['noisy_order_count'] = df_customer_sales['order_count'] + np.random.uniform(0, 0.01, size=len(df_customer_sales))
df_customer_sales['frequency_score'] = pd.qcut(df_customer_sales['noisy_order_count'], 5, labels=[1, 2, 3, 4, 5])
# 回购评分
df_customer_sales['recency_score'] = pd.qcut(df_customer_sales['last_order_days'], 5, labels=[5, 4, 3, 2, 1])

df_customer_sales.drop(columns=['noisy_order_count'], inplace=True)

# 忠诚度得分：综合 RFM 评分（可加权）
df_customer_sales['loyalty_score'] = df_customer_sales['recency_score'].astype(int) + \
                      df_customer_sales['frequency_score'].astype(int) + \
                      df_customer_sales['monetary_score'].astype(int)
df_customer_sales['loyalty_score'] = df_customer_sales['loyalty_score'] / 3
df_customer_sales['loyalty_score'] = df_customer_sales['loyalty_score'].round(1)

# 客户类型分类
df_customer_sales['tag'] = df_customer_sales.apply(classify_customer, axis=1)

customer_sort_by_sales = df_customer_sales['order_partner'].tolist()
df_customer_sales.head(10)



In [ ]:
segment_summary = df_customer_sales['tag'].value_counts().reset_index()
segment_summary.columns = ['Customer Segment', 'Count']
segment_summary

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 创建画布和子图布局
fig, axes = plt.subplots(2, 2, figsize=(16, 12))  # 2x2 子图布局

# 图 1: 饼图 - Customer Segments Distribution
axes[0, 0].pie(
    segment_summary['Count'], 
    labels=segment_summary['Customer Segment'], 
    autopct='%1.1f%%', 
    startangle=140
)

axes[0, 0].set_title('Verteilung der Kundenklassen', fontsize=14)


# 图 2: 销售额分布 - Distribution of Total Sales
sns.histplot(df_customer_sales['price_subtotal'], bins=20, kde=False, ax=axes[0, 1])
axes[0, 1].set_title('Verteilung des Gesamtumsatzes', fontsize=14)
axes[0, 1].set_xlabel('Gesamtumsatz - Netto (EUR)', fontsize=12)
axes[0, 1].set_ylabel('Anzahl der Kunden', fontsize=12)
axes[0, 1].grid(alpha=0.5)

# 图 3: 订单量分布 - Distribution of Total Orders
sns.histplot(df_customer_sales['order_count'], bins=20, kde=False, ax=axes[1, 0])
axes[1, 0].set_title('Verteilung der Gesamtbestellungen', fontsize=14)
axes[1, 0].set_xlabel('Auftragsmenge', fontsize=12)
axes[1, 0].set_ylabel('Anzahl der Kunden', fontsize=12)
axes[1, 0].grid(alpha=0.5)

# 图 4: 回购率分布 - Distribution of Days Since Last Order
sns.histplot(df_customer_sales['last_order_days'], bins=20, kde=False, ax=axes[1, 1])
axes[1, 1].set_title('Verteilung der Tage seit der letzten Bestellung', fontsize=14)
axes[1, 1].set_xlabel('Zuletzte Bestelltage', fontsize=12)
axes[1, 1].set_ylabel('Anzahl der Kunden', fontsize=12)
axes[1, 1].grid(alpha=0.5)

# 调整布局并保存图表
plt.tight_layout()
plt.savefig('data/sales_chart.png')  # 保存为 PNG 文件
plt.show()


散点图：
- 上方的点：该客户很可能已经流失
- 下方的点：该客户订购比较积极。
- 左方的点：该客户为小额客户。
- 右方的点：该客户为大额客户。
- 右上方的点：该客户是大额客户，但很可能已经流失。
- 左上方的点：该客户是小额客户，且很可能已经流失。
- 右下方的点：该客户是大额客户，且很可能是忠实客户。
- 若左半边的点很大：说明客户偏向于小额频繁订购。因此我们需要花费更多的运费。
- 若左下方的点很小：该客户很可能是新客户。

In [ ]:
import plotly.express as px
import pandas as pd

# 假设 df_customer_sales2 是一个 Pandas DataFrame
# df_customer_sales2['price_subtotal'], df_customer_sales2['last_order_days'], df_customer_sales2['order_count']

df_customer_sales2 = df_customer_sales.copy()  # 复制数据
# 准备数据
df_customer_sales2['size'] = df_customer_sales2['order_count'] * 100  # 点的大小
df_customer_sales2['label'] = df_customer_sales2['order_partner']  # 索引标签

# 使用 Plotly 创建交互式散点图
fig = px.scatter(
    df_customer_sales2,
    x='price_subtotal',  # 横坐标
    y='last_order_days',  # 纵坐标
    size='size',  # 点的大小
    color='order_count',  # 颜色映射
    hover_name='label',  # 鼠标悬停显示的内容
    hover_data={  # 添加其他悬停数据
        'avg_order_price': True,  # 显示 avg_order_price
        'order_count': ':.0f',  # 格式化 order_count 为整数
        'size': False,  # 不显示 size
        'tag': True  # 显示 tag
    },
    labels={
        'price_subtotal': 'Gesamt Netto (EUR)',        
        'order_count': 'Auftragsmenge',
        'avg_order_price': 'Durchschn. Auftragswert (EUR)',
        'last_order_days': 'Zuletzete Bestelltage',
        'tag': "Klasse"
    },
    title=f"Statistik zu Kundenbestellungen"
)

start_date = df_filtered_orderlines['create_date'].min().strftime('%d.%m.%Y')
end_date = df_filtered_orderlines['create_date'].max().strftime('%d.%m.%Y')
anno = f"Datenquelle: Odoo"
anno += f"<br>Zeitraum: {start_date} - {end_date}"
anno += f"<br>Anzahl Kunden: {len(df_customer_sales2)}"
anno += f"<br>Anzahl Aufträge: {df_customer_sales2['order_count'].sum()}"
anno += f"<br>Gesamtumsatz: {df_customer_sales2['price_subtotal'].sum():.2f} EUR"

# 设置布局
fig.update_layout(
    xaxis_title="Gesamtumsatz - Netto (EUR)",
    yaxis_title="Zuletzete Bestelltage",
    coloraxis_colorbar=dict(title="Auftragsmenge"), 
    template="plotly_white",
    width=1380,  # 图表的宽度（像素）
    height=500,  # 图表的高度（像素）
    annotations=[
        dict(
            text=anno,
            align="left",
            showarrow=False,
            xref="paper",  # 相对于图表的水平位置 (paper 表示相对整个图表区域)
            yref="paper",  # 相对于图表的垂直位置
            x=0.7,  # 靠近图表左侧
            y=0.9,  # 图表下方的适当位置（负值表示在图表下方）
            xanchor='left',  # 文本水平对齐方式
            yanchor='bottom',  # 文本垂直对齐方式
            font=dict(size=12, color="gray")  # 字体大小和颜色
        )
    ]
)

# 修改 y 轴刻度
fig.update_yaxes(
    tickvals=list(range(0, 300, 30))  # 设置刻度为 0, 30, 60, ..., 最大值可根据数据范围调整
)
# 保存为 HTML 文件
fig.write_html("data/customer_sales_bubble_chart.html")

# 显示图表
fig.show()


In [ ]:
# 统计每一个产品的销售额 (不含运费，不含为结的)
df_product_sales = odoo_sales_orderlines.groupby(['product_name']) \
                        .agg({'price_subtotal': sum, 
                              'currency': 'first', 
                              'product_uom_qty': sum,                              
                              'uom': 'first',
                              'order_number': 'nunique'})
df_product_sales = df_product_sales.rename(columns={'order_number': 'order_count'})

df_product_sales = df_product_sales.sort_values('price_subtotal', ascending=False)
# 计算上次购买时间（天）
last_order_days = odoo_sales_orderlines.groupby('product_name')['create_date'].max().reset_index()
last_order_days['last_order_days'] = (reference_date - last_order_days['create_date']).dt.days
df_product_sales = pd.merge(df_product_sales, last_order_days[['product_name', 'last_order_days']], on='product_name', how='left')


df_product_sales_top10 = df_product_sales.head(10)
df_product_sales_bottom10 = df_product_sales.tail(10)
product_sort_by_sales = df_product_sales.reset_index()['product_name'].tolist()
df_product_sales

In [ ]:
df_customer_month_sales = odoo_sales_orderlines.groupby(['order_partner', 'month']) \
    .agg({'price_subtotal': sum,
          'currency': 'first',
          'order_number': 'nunique'})

df_customer_month_sales = df_customer_month_sales.rename(columns={'order_number': 'order_count'})
df_customer_month_sales['avg_order_price'] = df_customer_month_sales['price_subtotal'] / df_customer_month_sales['order_count']
df_customer_month_sales['avg_order_price'] = df_customer_month_sales['avg_order_price'].round(2)

# 转换 month 列为 datetime 类型
df_customer_month_sales = df_customer_month_sales.reset_index()

# 排序：先按客户顺序，再按月份
df_customer_month_sales['customer_sort'] = pd.Categorical(
    df_customer_month_sales['order_partner'],  # 获取客户名
    categories=customer_sort_by_sales,  # 自定义排序规则
    ordered=True
)

df_customer_month_sales = df_customer_month_sales.sort_values(by=['customer_sort', 'month'])
df_customer_month_sales = df_customer_month_sales.drop(columns=['customer_sort'])

# 查看前 10 行
df_customer_month_sales.head(10)


In [ ]:
# 统计每个产品每个月卖出的数量和金额
df_product_month_sales = odoo_sales_orderlines.groupby(['product_name', 'month', ]) \
                            .agg({'price_subtotal': sum, 
                                  'currency': 'first', 
                                  'order_number': 'nunique'}) \
                          
df_product_month_sales = df_product_month_sales.rename(columns={'order_number': 'order_count'})
df_product_month_sales.sort_values(['product_name','month'], )
df_product_month_sales.reset_index(inplace=True)
df_product_month_sales.head(20)

In [ ]:
columns_map= {
        "order_partner": "Kunde",
        "price_subtotal": "Gesamt Netto (ERU)",
        "currency": "Währung",
        "order_count": "Auftragsmenge",
        "avg_order_price": "Durchschnittlicher Bestellpreis",
        "last_order_days": "Zuletzte Bestelltage",
        "frequency_score": "Häufigkeitsbewertung",
        "loyalty_score": "Treuepunktzahl",
        "tag": "Tag",
        "uom": "Maßeinheit",
        'product_uom_qty': 'Menge',
        "product_name": "Name",
        "month": "Monat",
    }

df_customer_sales = df_customer_sales.rename(columns= columns_map)
df_customer_month_sales = df_customer_month_sales.rename(columns=columns_map)
df_product_sales = df_product_sales.rename(columns=columns_map)
df_product_month_sales = df_product_month_sales.rename(columns=columns_map)

# Save in sheets
with pd.ExcelWriter('data/sales.xlsx') as writer:
    df_product_month_sales.to_excel(writer, sheet_name='product_month_sales')
    df_product_sales.to_excel(writer, sheet_name='product_sales', index=False)
    df_customer_month_sales.to_excel(writer, sheet_name='customer_month_sales', index=False)
    df_customer_sales.to_excel(writer, sheet_name='customer_sales', index=False)

4.1 细分客户群体
根据 RFM 得分，可以对客户进行分群：

高价值客户：忠诚度评分较高的客户（例如得分 ≥ 10）。
潜在流失客户：最近一次购买时间较久但曾经频繁购买的客户。
新客户：购买频率低但最近购买的客户。

# 获取Odoo产品信息

In [ ]:
product_ids = [line[1]['product_id'] for line in df_sale_order_lines.iterrows()]
product_ids = list(set(product_ids))
print(f"Number of unique products: {len(product_ids)}")

df_products = olib.fetch_all_product_template_details(product_ids)
print(f"Length of Product Details: {len(df_products)}")
df_products.to_excel("data/products.xlsx", index=False)
df_products.sample(3)


### **订单利润统计**
#### **统计方法**：
- **销售额（不含税）**：订单的客户支付总金额，不含税，包括产品价格和运费，字段为 `price_subtotal`。
- **订单成本（不含税）**：订单中所有产品的总成本，不含税，不包括运费，字段为 `order_cost`。默认值为 0 欧元。
- **运输成本**：根据订单中产品的总重量估算的运费成本，字段为 `estimated_shipping_cost`。默认值为 9.99 欧元。
- **订单利润（不含税）**：通过以下公式计算：

  $$
  \text{订单利润 (estimated\_profit)} = \text{销售额 (price\_subtotal)} - \text{订单成本 (order\_cost)} - \text{运输成本 (estimated\_shipping\_cost)}
  $$

#### **输出结果**：
生成包含每个订单利润明细的表格，输出表格名称为：`order_profit`。


### **Bestimmung des Auftragsgewinns**
#### **Berechnungsmethode**:
- **Umsatz (ohne Steuern)**: Der vom Kunden gezahlte Betrag ohne Steuern, einschließlich Produktpreis und Versandkosten. Feld: `price_subtotal`.
- **Auftragskosten (ohne Steuern)**: Die aktuellen Kosten der im Auftrag enthaltenen Produkte, ohne Versandkosten. Feld: `order_cost`. Standardwert: 0 Euro.
- **Versandkosten**: Die geschätzten Versandkosten basierend auf dem Produktgewicht. Feld: `estimated_shipping_cost`. Standardwert: 9,99 Euro.
- **Auftragsgewinn (ohne Steuern)**: Berechnet mit der folgenden Formel:

  $$
  \text{Auftragsgewinn (estimated\_profit)} = \text{Umsatz (price\_subtotal)} - \text{Auftragskosten (order\_cost)} - \text{Versandkosten (estimated\_shipping\_cost)}
  $$

#### **Ergebnis**:
Erstellung einer Tabelle mit detaillierten Auftragsgewinnen. Tabellenname: `order_profit`.

In [ ]:
df_exact = odoo_sales_orderlines.merge(df_products[["id", "display_name", "default_code", "standard_price", "weight", "categ_id"]], \
                                       left_on='product_id', right_on='id', how="left")

df_sales_orderlines_combined = df_exact.copy()
df_sales_orderlines_combined.loc[df_sales_orderlines_combined['product_name'] == 'Versandkostenpauschale', 'standard_price'] = 0
df_sales_orderlines_combined['total_weight'] = df_sales_orderlines_combined['product_uom_qty'] * df_sales_orderlines_combined['weight']
df_sales_orderlines_combined['order_cost'] = df_sales_orderlines_combined['product_uom_qty'] * df_sales_orderlines_combined['standard_price']

df_sales_orderlines_combined.to_excel("data/sales_orderlines_combined.xlsx", index=False)
df_sales_orderlines_combined.sample(5)


In [ ]:
# 按订单号聚合订单明细.
df_agg_orderlines = df_sales_orderlines_combined.groupby('order_number')  \
    .agg({'create_date': 'first', 'product_name': 'nunique',  
          'order_partner': 'first', 'salesman': 'first', 'total_weight': 'sum',
          'price_tax': 'sum',  'price_subtotal': 'sum', 'order_cost': 'sum', 'currency': 'first'}) \
    .rename(columns={'product_name': 'line_count'})  \
    .reset_index()
print(f"Length: {len(df_agg_orderlines)}")

df_agg_orderlines['estimated_shipping_cost'] = 9.99

df_agg_orderlines['estimated_profit'] = df_agg_orderlines['price_subtotal'] - df_agg_orderlines['order_cost'] - df_agg_orderlines['estimated_shipping_cost']

sales_order_combined = df_agg_orderlines.copy()

# sales_order_combined.to_excel("data/sales_order_combined.xlsx", index=False)
sales_order_combined.sample(3)

In [ ]:
print("缺失数据")
cond = (df_sales_orderlines_combined['total_weight'] == 0) | (df_sales_orderlines_combined['total_weight'].isna())
df_sales_orderlines_combined_weight0 = df_sales_orderlines_combined[cond]
df_sales_orderlines_combined_weight0 = df_sales_orderlines_combined_weight0.groupby('product_name') \
    .agg({'internal_reference': 'first', 'total_weight': 'last', 'uom': 'last'}) \
    .reset_index()  
print("Lenght of sales_order_combined_weight0: ", len(df_sales_orderlines_combined_weight0))

cond = df_sales_orderlines_combined['standard_price'] == 0
df_sales_orderlines_combined_price0 = df_sales_orderlines_combined[cond]
df_sales_orderlines_combined_price0 = df_sales_orderlines_combined_price0.groupby('product_name') \
    .agg({'internal_reference': 'first', 'standard_price': 'last', 'uom': 'last'}) \
    .reset_index()  
print("Lenght of sales_order_combined_price0: ", len(df_sales_orderlines_combined_price0))



### **客户利润统计**
#### **统计方法**：
- 按客户聚合订单利润表（`order_profit`），计算每位客户的总利润。
#### **输出结果**：
生成包含每位客户总利润的表格，输出表格名称为：`customer_profit`。

### **Bestimmung des Kundengewinns**
#### **Berechnungsmethode**:
- Aggregation der Auftragsgewinntabelle (`order_profit`), um den Gesamtgewinn jedes Kunden zu berechnen.

#### **Ergebnis**:
Erstellung einer Tabelle mit dem Gewinn jedes Kunden. Tabellenname: `customer_profit`.


In [ ]:
# 统计每个客户的利润
df_customer_sales_profit = sales_order_combined.groupby('order_partner') \
                        .agg({'price_subtotal': sum, 
                              'salesman': 'first',
                              'order_cost': sum, 
                              'estimated_shipping_cost': sum, 
                              'estimated_profit': sum, 
                              'currency': 'first',                                    
                              'order_number': 'nunique'}) \
                        .reset_index() 

df_customer_sales_profit['profit_rate'] = df_customer_sales_profit['estimated_profit'] / df_customer_sales_profit['order_cost']
df_customer_sales_profit['profit_rate'] = df_customer_sales_profit['profit_rate'].round(2)

with pd.ExcelWriter('data/customer_sales_profit.xlsx') as writer:
    sales_order_combined.to_excel(writer, sheet_name="order_profit")
    df_customer_sales_profit.to_excel(writer, sheet_name="customer_profit")
    df_sales_orderlines_combined_price0.to_excel(writer, sheet_name="no_cost")
    df_sales_orderlines_combined_weight0.to_excel(writer, sheet_name="no_weight")

df_customer_sales_profit.sample(4)

### **产品利润统计**
#### **统计方法**：
- **产品总销售额（不含税）**：产品历史总销售额的累计金额，不含税，字段为 `price_subtotal`。
- **产品销售总成本（不含税）**：产品历史总销售成本的累计金额，不含税，字段为 `order_cost`。
- **产品利润（不含税）**：通过以下公式计算：
  $$
  \text{产品利润 (profit)} = \text{产品总销售额 (price\_subtotal)} - \text{产品销售总成本 (order\_cost)}
  $$

#### **输出结果**：
生成包含每个产品利润明细的表格，输出表格名称为：`product_profit`。


### **Bestimmung des Produktgewinns**
#### **Berechnungsmethode**:
- **Gesamtumsatz des Produkts (ohne Steuern)**: Der kumulierte Gesamtumsatz des Produkts (ohne Steuern). Feld: `price_subtotal`.
- **Gesamtkosten des Produkts (ohne Steuern)**: Die aktuellen kumulierten Kosten des Produkts (ohne Steuern). Feld: `order_cost`.
- **Produktgewinn (ohne Steuern)**: Berechnet mit der folgenden Formel:

  $$
  \text{Produktgewinn (profit)} = \text{Gesamtumsatz (price\_subtotal)} - \text{Gesamtkosten (order\_cost)}
  $$

#### **Ergebnis**:
Erstellung einer Tabelle mit den Gewinnen jedes Produkts. Tabellenname: `product_profit`.

In [ ]:
print("缺失数据")
cond = df_sales_orderlines_combined['categ_id'] == "All"
df_sales_orderlines_combined_categ0 = df_sales_orderlines_combined[cond]

df_sales_orderlines_combined_categ0 = df_sales_orderlines_combined_categ0.groupby('product_name') \
    .agg({'internal_reference': 'first', 'categ_id': 'first'}) \
    .reset_index()

df_sales_orderlines_combined_categ0.sample(3)

In [ ]:
# 按产品聚合订单明细.
df_agg_orderlines = df_sales_orderlines_combined.groupby('product_name')  \
    .agg({'internal_reference': 'last','categ_id': 'last',  
          'total_weight': 'sum', 'product_name': 'count', 
          'price_tax': 'sum',  'price_subtotal': 'sum',           
          'order_cost': 'sum', 'currency': 'first'}) \
    .rename(columns={'product_name': 'line_count'})  \
    .reset_index()

df_agg_orderlines.sort_values(by='product_name', ascending=True, inplace=True)
df_agg_orderlines = df_agg_orderlines[df_agg_orderlines['product_name'] != 'Versandkostenpauschale']

df_agg_orderlines['estimated_profit'] = df_agg_orderlines['price_subtotal'] - df_agg_orderlines['order_cost']
df_agg_orderlines['profit_rate'] = df_agg_orderlines['estimated_profit']/df_agg_orderlines['order_cost']
df_agg_orderlines['profit_rate'] = df_agg_orderlines['profit_rate'].round(2)

print(f"Length: {len(df_agg_orderlines)}")

df_product_sales_profit = df_agg_orderlines
df_product_sales_profit.head(3)

### **产品分类利润统计**
#### **统计方法**：
- **产品分类总销售额（不含税）**：按产品分类统计历史总销售额的累计金额，不含税，字段为 `price_subtotal`。
- **产品分类销售总成本（不含税）**：按产品分类统计历史总销售成本的累计金额，不含税，字段为 `order_cost`。
- **产品分类利润（不含税）**：通过以下公式计算：
  $$
  \text{产品分类利润 (profit)} = \text{产品分类总销售额 (price\_subtotal)} - \text{产品分类销售总成本 (order\_cost)}
  $$
#### **输出结果**：
生成包含每个产品分类利润明细的表格，输出表格名称为：`category_profit`。


### **Bestimmung des Gewinns nach Produktkategorien**
#### **Berechnungsmethode**:
- **Gesamtumsatz der Produktkategorie (ohne Steuern)**: Der kumulierte Gesamtumsatz aller Produkte in einer Kategorie (ohne Steuern). Feld: `price_subtotal`.
- **Gesamtkosten der Produktkategorie (ohne Steuern)**: Die aktuellen kumulierten Kosten aller Produkte in einer Kategorie (ohne Steuern). Feld: `order_cost`.
- **Gewinn der Produktkategorie (ohne Steuern)**: Berechnet mit der folgenden Formel:

  $$
  \text{Gewinn der Kategorie (profit)} = \text{Gesamtumsatz (price\_subtotal)} - \text{Gesamtkosten (order\_cost)}
  $$

#### **Ergebnis**:
Erstellung einer Tabelle mit den Gewinnen jeder Produktkategorie. Tabellenname: `category_profit`.

In [ ]:
# 按产品分类聚合订单明细.
df_agg_orderlines = df_sales_orderlines_combined.groupby('categ_id')  \
    .agg({'product_name': 'last', 'internal_reference': 'last',  
          'total_weight': 'sum', 'categ_id': 'count', 
          'price_tax': 'sum',  'price_subtotal': 'sum', 
          'order_cost': 'sum', 'currency': 'first'}) \
        .rename(columns={'categ_id': 'line_count'})  \
        .reset_index()

df_agg_orderlines.sort_values(by='product_name', ascending=True, inplace=True)
df_agg_orderlines = df_agg_orderlines[df_agg_orderlines['product_name'] != 'Versandkostenpauschale']

df_agg_orderlines['estimated_profit'] = df_agg_orderlines['price_subtotal'] - df_agg_orderlines['order_cost']
df_agg_orderlines['profit_rate'] = df_agg_orderlines['estimated_profit']/df_agg_orderlines['order_cost']
df_agg_orderlines['profit_rate'] = df_agg_orderlines['profit_rate'].round(2)

print(f"Length: {len(df_agg_orderlines)}")

df_product_categ_sales_profit = df_agg_orderlines
df_product_categ_sales_profit.head(3)

In [ ]:
with pd.ExcelWriter('data/product_sales_profit.xlsx') as writer:
    df_product_sales_profit.to_excel(writer, sheet_name='product_profit')
    df_product_categ_sales_profit.to_excel(writer, sheet_name='category_profit')
    df_sales_orderlines_combined_categ0.to_excel(writer, sheet_name='no_category')

# 获得采购成本

### 获得所有的Bestellung


In [ ]:
df_purchase_orders, orderline_ids = olib.fetch_all_purchase_order_details()
print(f"Length of purchase orders: {len(df_purchase_orders)}")
print(f"Length of orderline_ids: {len(orderline_ids)}")
df_purchase_orders.to_excel("data/purchase_orders.xlsx", index=False)
df_purchase_orders.sample(5)

In [ ]:
df_purchase_order_lines = olib.fetch_all_purchase_orderline_details(orderline_ids)
print(f"Length of purchase order lines: {len(df_purchase_order_lines)}")
df_purchase_order_lines.to_excel('data/purchase_order_lines.xlsx')

In [ ]:
import datetime
# 数据清洗

# 被取消的orderlines
df_filtered_purchase_orderlines = df_purchase_order_lines[df_purchase_order_lines['state']== 'purchase']

# product_type == product
df_filtered_purchase_orderlines = df_filtered_purchase_orderlines[df_filtered_purchase_orderlines['product_type'] == 'product']

# 订单中所有商品的qty_received == product_uom_qty
qty_all_received = df_filtered_purchase_orderlines['qty_received'] == df_filtered_purchase_orderlines['product_uom_qty']
# df_filtered_purchase_orderlines = df_filtered_purchase_orderlines[qty_all_received]

# price_unit != 0
df_filtered_purchase_orderlines = df_filtered_purchase_orderlines[df_filtered_purchase_orderlines['price_unit']!= 0]

df_filtered_purchase_orderlines['year'] = df_filtered_purchase_orderlines['create_date'].dt.strftime('%Y')
df_filtered_purchase_orderlines['month'] = df_filtered_purchase_orderlines['create_date'].dt.strftime('%Y-%m')

df_filtered_purchase_orderlines.to_excel('data/purchase_order_lines.xlsx')
print(len(df_filtered_purchase_orderlines))

In [ ]:
# 统计出每个产品的采购价
odoo_purchase_orderlines = df_filtered_purchase_orderlines.copy()


df_product_cost = odoo_purchase_orderlines.groupby(['product_name']) \
                            .agg({'price_unit': ['mean', 'first', 'last', 'min', 'max'], 
                                  'currency': 'first', 
                                  'product_name':  'count',
                                  'product_uom_qty': 'sum'}) 

df_product_cost.columns = ['_'.join(col).strip() for col in df_product_cost.columns]
print(len(df_product_cost))
df_product_cost.reset_index(inplace=True)
df_product_cost.sample(3)